<a href="https://colab.research.google.com/github/RiyaMaurya1/Deep_learning/blob/main/skin_cancer_using_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["KAGGLE_USERNAME"] = "riyaasat"
os.environ["KAGGLE_KEY"] = "551f8333d17ea1d826c72c8ad68f1ed4"

In [ ]:
!kaggle datasets download nodoubttome/skin-cancer9-classesisic

 99% 778M/786M [00:06<00:00, 83.1MB/s]
100% 786M/786M [00:06<00:00, 134MB/s] 


In [ ]:
from IPython.display import clear_output
! unzip skin-cancer9-classesisic.zip
clear_output()

In [ ]:
%load_ext tensorboard

In [ ]:
import cv2
# to see image without any show method along with wait.
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.applications import InceptionV3, VGG19, ResNet101

In [ ]:
train_path = "dataset/Train"
test_path = "dataset/Test"


In [ ]:
vgg = VGG19()

In [ ]:
image_data_gen = ImageDataGenerator(horizontal_flip=True,
                                    rescale=1/255.,
                                    rotation_range=30,
                                    vertical_flip=True
                                    )

In [ ]:
X_train = image_data_gen.flow_from_directory(train_path, target_size=(224,224), batch_size=32, class_mode="sparse")


Found 2239 images belonging to 9 classes.


In [ ]:
X_val = image_data_gen.flow_from_directory(test_path, target_size=(224,224), batch_size=4, class_mode="sparse")

Found 118 images belonging to 9 classes.


In [ ]:
vgg.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
vgg.output

<KerasTensor: shape=(None, 1000) dtype=float32 (created by layer 'predictions')>

In [ ]:
x = vgg.output
# Add output layer
output = Dense(9, activation = "softmax")(x)

In [ ]:
model = Model(inputs = vgg.input, outputs = output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from tensorflow.python.ops import histogram_ops
from datetime import datetime as dt
def train_model():
  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  logdir = os.path.join("logs", dt.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(X_train, validation_data=X_val, epochs=50, callbacks=[tensorboard_callback])

In [ ]:
train_model()

Epoch 1/50
70/70 [==============================] - ETA: 0s - loss: 2.1492 - accuracy: 0.2019